In [602]:
import numpy as np
from typing import Optional
import numpy.typing as npt
from collections import defaultdict
import hashlib

with open('./assets/input_day_17.txt') as file:
    jets = list(file.read())

In [611]:
class Shape(object):
    def __init__(self,indices) -> None:
        self._indices = indices
    
    @property
    def h(self):
        return max(x[1] for x in self._indices) - min(x[1] for x in self._indices) + 1


    def __add__(self,o: tuple):
        return tuple([(x[0]+o[0],x[1]+o[1]) for x in self._indices])

    def getd(self, direction):
        match direction:
            case ">":
                return (1, 0)
            case "<":
                return (-1, 0)
            case "v":
                return (0, 1)
            case "^":
                return (0, -1)


    def move_idx(self,direction):
        return set(self + self.getd(direction)) - set(self._indices)

    def move(self, direction):
        self._indices = self + self.getd(direction)


    def move_allowed(self,direction,space):
        idx = self.move_idx(direction)
        if any([x[0] < 0 or x[0] > space.shape[1] -1 for x in idx]):
            return False
        elif any([x[1] < 0 or x[1] > space.shape[0] -1 for x in idx]):
            return False
        elif any([space[x[::-1]] for x in idx]):
            return False
        else:
            return True


class Cave(object):

    SHAPES = [
        ((0,0),(1,0),(2,0),(3,0)),
        ((1,0),(0,1),(1,1),(2,1),(1,2)),
        ((2,0),(2,1),(0,2),(1,2),(2,2)),
        ((0,0),(0,1),(0,2),(0,3)),
        ((0,0),(0,1),(1,0),(1,1))
    ]

    def __init__(self, input, start_x = 2, w = 7) -> None:
        self.start_x = start_x
        self.w = w
        self.stopped = 0
        self.jet_id = 0
        self.shape_id = 0
        self.space = np.zeros((0,self.w),dtype=bool)
        self.jets = input
        self.hashes = {}

    def signature(self, n=30):
        return frozenset(
            (
                hashlib.sha1(self.space[0:min(n,self.space.shape[0]),:].view(np.uint8)).hexdigest(),
                self.shape_id
            )
        )

    def drop(self):
        
        hs = self.signature()
        loop = None, None

        if hs in self.hashes:
            loop = (self.space.shape[0] - self.hashes[hs][0], self.stopped - self.hashes[hs][1])
            # return hit
            # else:
        self.hashes[hs] = (self.space.shape[0],self.stopped)

        shape = Shape(self.SHAPES[self.shape_id])
        
        dead_space = np.zeros((3 + shape.h,self.w),dtype=bool)
        self.space = np.concatenate([dead_space,self.space])

        # move to starting position
        for _ in range(self.start_x):
            if shape.move_allowed(">",self.space):
                shape.move(">")
        
        while True:
            if shape.move_allowed(self.jets[self.jet_id],self.space):
                shape.move(self.jets[self.jet_id])

            self.jet_id = (self.jet_id + 1) % len(self.jets)

            if shape.move_allowed("v", self.space):
                shape.move("v")
            else:
                c,r  = list(zip(*shape._indices))
                self.space[r,c] = True
                self.stopped += 1
                self.shape_id = (self.shape_id + 1) % len(self.SHAPES)
                front = list(range(np.argmax(np.sum(self.space,axis=1) > 0)))
                self.space = np.delete(self.space,front,axis=0)
                break
        
        return loop


In [612]:
pt1 = Cave(jets)
for _ in range(2022):
    hit = pt1.drop()

pt1.space.shape[0]

3137

In [613]:
pt2 = Cave(jets)
while True:
    pattern = pt2.drop()
    if pattern[0] is not None:
        break

n = 1_000_000_000_000
rem = n - pt2.stopped
loops = rem // pattern[1]
more_rocks = rem % pattern[1]

for _ in range(more_rocks):
    pt2.drop()

loops * pattern[0] + pt2.space.shape[0]

1564705882327